In [ ]:
#@title Celestial Empire — Check Savedata from Engine
from pathlib import Path
import zlib, re

In [ ]:


try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

SIG = b"\x78\x9C"

FEST_BP  = b'/Game/ChinaBuilder/Gameplay/Festivals/Bp_Festivals.Bp_Festivals_C'
FERT_BP  = b'/Game/ChinaBuilder/Gameplay/Fertility/Bp_Fertility.Bp_Fertility_C'

REPS_MINIMAL = [
    (b"BP_MonBonus_Festivals_C", b"BP_MonBonus_Fertility_C"),
    (b"BP_MonBonus_Festivals",   b"BP_MonBonus_Fertility"),
    (FEST_BP, FERT_BP),
]
REPS_SAFE_PLUS = REPS_MINIMAL + [
    (b"EFestivalStatus::Active", b"EFestivalStatus::Disabled"),
]

TOKS = {
    "fert": b"BP_MonBonus_Fertility",
    "fert_c": b"BP_MonBonus_Fertility_C",
    "fest": b"BP_MonBonus_Festivals",
    "fest_c": b"BP_MonBonus_Festivals_C",
    "active": b"EFestivalStatus::Active",
    "disabled": b"EFestivalStatus::Disabled",
    "bpf": FEST_BP,
    "bpfert": FERT_BP,
    "mon": b"CurrentMonumentBonusesSave",
    "festivals_state": b"FestivalsState",
    "UnlockedBuildings": b"UnlockedBuildings",
}

COMPOSTER    = b"Building.Production.Composter"
REPAIRMANHUT = b"Building.Special.RepairmanHut"
assert len(COMPOSTER) == len(REPAIRMANHUT) == 29

def upload_once(prompt):
  print(prompt)
  return files.upload() if IN_COLAB else {}

def find_chunks(blob: bytes):
  chunks = []
  pos = 0
  while True:
    i = blob.find(SIG, pos)
    if i == -1: break
    d = zlib.decompressobj()
    out = d.decompress(blob[i:])
    used = len(blob[i:]) - len(d.unused_data)
    if used <= 0:
      pos = i + 2; continue
    comp = blob[i:i+used]
    chunks.append((i, used, out, comp))
    pos = i + used
  return chunks

def count_tokens(blob: bytes):
  chunks = find_chunks(blob)
  c = {k:0 for k in TOKS}
  for _,_,out,_ in chunks:
    for k,v in TOKS.items():
      c[k] += out.count(v)
  return len(chunks), c

def patch_chunkwise(blob: bytes, rep_map):
  chunks = find_chunks(blob)
  out_bytes = bytearray(); pos=0
  changed=[]
  for off,used,out,comp in chunks:
    out_bytes += blob[pos:off]
    new_out = out
    hits = 0
    for src,dst in rep_map:
      cnt = new_out.count(src)
      if cnt:
        hits += cnt
        new_out = new_out.replace(src,dst)
    if hits:
      new_comp = zlib.compress(new_out)
      out_bytes += new_comp
      changed.append((off, len(comp), len(new_comp)))
    else:
      out_bytes += comp
    pos = off + used
  out_bytes += blob[pos:]
  return bytes(out_bytes), changed

def find_unlocked_buildings_chunk(blob: bytes):
  for off,used,out,comp in find_chunks(blob):
    if b"UnlockedBuildings" in out:
      return off, used, out, comp
  return None

def swap_repairman_to_composter_in_unlocks(blob: bytes):
  chunks = find_chunks(blob)
  out_bytes = bytearray(); pos=0
  did_swap=False; info=None

  for off,used,out,comp in chunks:
    out_bytes += blob[pos:off]
    if TOKS["UnlockedBuildings"] in out and COMPOSTER not in out:
      if REPAIRMANHUT in out:
        new_out = out.replace(REPAIRMANHUT, COMPOSTER, 1)
        out_bytes += zlib.compress(new_out)
        did_swap=True
        info=("UnlockedBuildings chunk", off, len(comp))
      else:
        out_bytes += comp
    else:
      out_bytes += comp
    pos = off + used

  out_bytes += blob[pos:]
  return bytes(out_bytes), did_swap, info

up1 = upload_once("👉 Upload your main Level.sav (required)")
if not up1:
  raise SystemExit("No files uploaded.")
main_name = None
for k in up1:
  if k.lower() == "level.sav": main_name = k; break
if not main_name:
  main_name = next((k for k in up1 if k.lower().endswith(".sav")), None)
if not main_name:
  raise SystemExit("No .sav provided.")

orig = up1[main_name]
Path(main_name).write_bytes(orig)
print(f"Saved: {main_name}  ({len(orig):,} bytes)")

fert_bytes = None
try:
  up2 = upload_once("👉 Upload Level_fert_test.sav (optional); else Cancel/ESC")
  if up2:
    name2 = None
    for k in up2:
      if k.lower() == "level_fert_test.sav": name2 = k; break
    if not name2:
      name2 = next((k for k in up2 if k.lower().endswith(".sav")), None)
    if name2:
      fert_bytes = up2[name2]
      Path(name2).write_bytes(fert_bytes)
      print(f"Saved: {name2}  ({len(fert_bytes):,} bytes)")
except Exception:
  print("Second upload skipped.")

n0,c0 = count_tokens(orig)
print("\n--- Original ---")
print("Chunks:", n0, " Size:", len(orig))
for k in ["fert","fert_c","fest","fest_c","active","disabled","bpf","bpfert","mon","festivals_state"]:
  print(f"{k:16s}:", c0[k])

min_blob, min_changed = patch_chunkwise(orig, REPS_MINIMAL)
n1,c1 = count_tokens(min_blob)
print("\n--- MINIMAL ---")
print("Chunks:", n1, " Size:", len(min_blob))
for k in ["fert","fert_c","fest","fest_c","active","disabled","bpf","bpfert","mon","festivals_state"]:
  print(f"{k:16s}:", c1[k])
print("MINIMAL changed chunks:", min_changed)

fix_blob, did_swap, info = swap_repairman_to_composter_in_unlocks(min_blob)
n2,c2 = count_tokens(fix_blob)
print("\n--- MINIMAL + Composter unlock ---")
print("Chunks:", n2, " Size:", len(fix_blob))
for k in ["fert","fert_c","fest","fest_c","active","disabled","bpf","bpfert","mon","festivals_state"]:
  print(f"{k:16s}:", c2[k])
print("Did swap RepairmanHut→Composter?", did_swap, " Info:", info)

safe_blob, safe_changed = patch_chunkwise(orig, REPS_SAFE_PLUS)
safe_blob2, did_swap2, info2 = swap_repairman_to_composter_in_unlocks(safe_blob)
n3,c3 = count_tokens(safe_blob2)
print("\n--- SAFE+ + Composter unlock ---")
print("Chunks:", n3, " Size:", len(safe_blob2))
for k in ["fert","fert_c","fest","fest_c","active","disabled","bpf","bpfert","mon","festivals_state"]:
  print(f"{k:16s}:", c3[k])
print("SAFE+ changed chunks:", safe_changed, "  Did swap?", did_swap2, " Info:", info2)

min_name   = "Level_patched_MINIMAL_fertility_with_composter.sav"
safe_name  = "Level_patched_SAFEPLUS_fertility_with_composter.sav"
Path(min_name).write_bytes(fix_blob)
Path(safe_name).write_bytes(safe_blob2)
print(f"\nWrote:\n- {min_name}  ({len(fix_blob):,} bytes)\n- {safe_name}  ({len(safe_blob2):,} bytes)")

if IN_COLAB:
  try:
    print("\n⬇️  Download MINIMAL+Composter…")
    files.download(min_name)
  except Exception:
    print(f"Use Files panel → {min_name}")
  try:
    print("\n⬇️  Download SAFE+Composter…")
    files.download(safe_name)
  except Exception:
    print(f"Use Files panel → {safe_name}")


👉 Upload your main Level.sav (required)


Saving Level.sav to Level (4).sav
Saved: Level (4).sav  (12,931,617 bytes)
👉 Upload Level_fert_test.sav (optional); else Cancel/ESC


Saving Level_fert_test.sav to Level_fert_test (1).sav
Saved: Level_fert_test (1).sav  (13,093,287 bytes)

--- Original ---
Chunks: 228  Size: 12931617
fert            : 0
fert_c          : 0
fest            : 2
fest_c          : 1
active          : 2
disabled        : 7
bpf             : 1
bpfert          : 0
mon             : 1
festivals_state : 1

--- MINIMAL ---
Chunks: 228  Size: 12931622
fert            : 2
fert_c          : 1
fest            : 0
fest_c          : 0
active          : 2
disabled        : 7
bpf             : 0
bpfert          : 1
mon             : 1
festivals_state : 1
MINIMAL changed chunks: [(12928009, 3608, 3613)]

--- MINIMAL + Composter unlock ---
Chunks: 228  Size: 12931624
fert            : 2
fert_c          : 1
fest            : 0
fest_c          : 0
active          : 2
disabled        : 7
bpf             : 0
bpfert          : 1
mon             : 1
festivals_state : 1
Did swap RepairmanHut→Composter? True  Info: ('UnlockedBuildings chunk', 12718553, 15285)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


⬇️  Download SAFE+Composter…


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>